<a href="https://colab.research.google.com/github/dickyadem/Times-Series-with-LSTM-MAE-under-10-/blob/main/fixTimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv('/content/weatherHistory.csv')
df.head()

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [ ]:
df.isnull().sum()

Formatted Date                0
Summary                       0
Precip Type                 517
Temperature (C)               0
Apparent Temperature (C)      0
Humidity                      0
Wind Speed (km/h)             0
Wind Bearing (degrees)        0
Visibility (km)               0
Loud Cover                    0
Pressure (millibars)          0
Daily Summary                 0
dtype: int64

In [ ]:
data = df.filter(['Temperature (C)'])

dataset = data.values 
training_data_len = int(np.ceil( len(dataset) * .8 ))

training_data_len

77163

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

array([[0.50697507],
       [0.50508505],
       [0.50544505],
       ...,
       [0.71055711],
       [0.70218702],
       [0.68463685]])

In [ ]:
train = train_test_split(scaled_data, test_size=0.2, shuffle=False)
train_data = scaled_data[0:int(training_data_len), :]
x_train = []
y_train = []
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<= 61:
        print(x_train)
        print(y_train)
        print()
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

[array([0.50697507, 0.50508505, 0.50544505, 0.48780488, 0.49536495,
       0.50292503, 0.47880479, 0.49563496, 0.52884529, 0.57663577,
       0.61299613, 0.63126631, 0.64188642, 0.63432634, 0.65934659,
       0.6598866 , 0.60282603, 0.60543605, 0.58446584, 0.56646566,
       0.54063541, 0.53469535, 0.51741517, 0.51876519, 0.52236522,
       0.51408514, 0.53469535, 0.46944469, 0.45252453, 0.46350464,
       0.47115471, 0.4798848 , 0.51345513, 0.55152552, 0.59805598,
       0.63468635, 0.66141661, 0.67824678, 0.69453695, 0.69669697,
       0.67941679, 0.68103681, 0.67752678, 0.64188642, 0.61371614,
       0.59688597, 0.58716587, 0.58446584, 0.57663577, 0.56871569,
       0.49338493, 0.53577536, 0.53469535, 0.52677527, 0.53352534,
       0.53352534, 0.55062551, 0.5601656 , 0.57771578, 0.61569616])]
[0.6372963729637298]

[array([0.50697507, 0.50508505, 0.50544505, 0.48780488, 0.49536495,
       0.50292503, 0.47880479, 0.49563496, 0.52884529, 0.57663577,
       0.61299613, 0.63126631, 0.641

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.LSTM(60),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
])

In [ ]:
class my_allback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('mae') < 0.1):
            print("MAE sudah dibawah 10%")

    def on_train_end(self, epoch, logs={}):
        print('Done')

callbacks = my_allback()

In [ ]:
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=["mae"])
history = model.fit(x_train, y_train,
                    epochs=1,
                    batch_size=256,  callbacks = callbacks)

302/302 [==============================] - 85s 266ms/step - loss: 0.0044 - mae: 0.0664
Done
